In [0]:
from pyspark.sql import SparkSession, DataFrame
from typing import Dict, Optional, List
from pyspark.sql.types import StructType

In [0]:
# --- Clase para la conexión y lectura de tablas ---
class PostgresReader:
    def __init__(
        self,
        scope: str,
        username_key: str,
        password_key: str,
        hostname: str,
        port: int,
        database: str,
        driver: str = "org.postgresql.Driver",
    ):
        self.jdbc_username = dbutils.secrets.get(scope=scope, key=username_key)
        self.jdbc_password = dbutils.secrets.get(scope=scope, key=password_key)
        self.jdbc_url = (
            f"jdbc:postgresql://{hostname}:{port}/{database}?sslmode=require"
        )
        self.connection_properties: Dict[str, str] = {
            "user": self.jdbc_username,
            "password": self.jdbc_password,
            "driver": driver,
        }


    def read_table(self, table_name: str, schema: Optional[StructType] = None) -> DataFrame:
        df = spark.read.jdbc(
            url=self.jdbc_url, table=table_name, properties=self.connection_properties
        )

        if schema is not None:
            for field in schema.fields:
                df = df.withColumn(field.name, df[field.name].cast(field.dataType))

        return df

In [0]:
# --- Clase para la escritura de tablas ---
class DeltaWriter:
    def __init__(self, base_path: str = "/mnt/bronze"):
        self.base_path = base_path


    def write(
        self,
        df: DataFrame,
        table_name: str,
        database: str = "bronze",
        partition_cols: Optional[List[str]] = None,
        mode: str = "overwrite",
    ):

        write_path = f"{self.base_path}/{table_name}"
        full_table_name = f"{database}.{table_name}"
        writer = df.write.format("delta").mode(mode)

        if partition_cols:
            writer = writer.partitionBy(*partition_cols)

        writer.save(write_path)

        spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
        spark.sql(
            f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{write_path}'
        """
        )

        print(f"✅ Datos escritos y tabla registrada como: {full_table_name}")